## 任务如何取消

<font color="pink">CancellationTokenSource + CancellationToken</font>


CTS实现了IDisposable接口，所以需要释放

In [3]:
using System.Diagnostics;
using System.Threading;

var cts = new CancellationTokenSource();
var token = cts.Token;

var sw = Stopwatch.StartNew();
try
{
    var cancelTask = Task.Run(async () => {
        await Task.Delay(3000);
        cts.Cancel();
    });
    await Task.WhenAll(Task.Delay(10000,token),cancelTask);
}
catch(TaskCanceledException ex)
{
    Console.WriteLine("TaskCanceled");
}finally
{
    cts.Dispose();
}

Console.WriteLine($"Task completed in {sw.ElapsedMilliseconds}ms");

TaskCanceled
Task completed in 3012ms


CTS还可以传入一个TimeSpan，表示超时后自动取消，或调用CancelAfter()方法

In [4]:
using System.Diagnostics;
using System.Threading;

var cts = new CancellationTokenSource(TimeSpan.FromSeconds(3));
var token = cts.Token;

var sw = Stopwatch.StartNew();
try
{
    await Task.Delay(10000,token);
}
catch(TaskCanceledException ex)
{
    Console.WriteLine("TaskCanceled");
}finally
{
    cts.Dispose();
}

Console.WriteLine($"Task completed in {sw.ElapsedMilliseconds}ms");

TaskCanceled
Task completed in 3007ms


In [5]:
using System.Diagnostics;
using System.Threading;

var cts = new CancellationTokenSource();
cts.CancelAfter(TimeSpan.FromMilliseconds(3000));
var token = cts.Token;

var sw = Stopwatch.StartNew();
try
{
    await Task.Delay(10000,token);
}
catch(TaskCanceledException ex)
{
    Console.WriteLine("TaskCanceled");
}finally
{
    cts.Dispose();
}

Console.WriteLine($"Task completed in {sw.ElapsedMilliseconds}ms");

TaskCanceled
Task completed in 3005ms


编写支持取消的方法

cts.Cancel() & Token.IsCancellationRequest
- Token.ThrowIdCancelllationRequested()

In [ ]:
async Task FooAsync(CancellationToken? cancellationToken = null)
{
    var token = cancellationToken ?? CancellationToken.None;
    await Task.Delay(1000,token);
}


Task FooAsync3(CancellationToken cancellationToken)
{
    return Task.Run(() => {
        if(cancellationToken.IsCancellationRequested)
        {
            cancellationToken.ThrowIfCancellationRequested();
        }
        //判断是否被取消
        while(!cancellationToken.IsCancellationRequested)
        {
            Thread.Sleep(1000);
            Console.WriteLine("Pooling...");
        }
    });
}

### 任务取消时的对策

1. 抛出异常
    - OperationCanceledException & TaskCanceledException
2. 提前返回
    - Task.FromCancelled
3. 记得善后
    - try catch finally
    - Token.Register()

In [ ]:
Task<string> FooAsync4(CancellationToken cancellationToken)
{
    var task = new Task(() => {});

    if(cancellationToken.IsCancellationRequested)
        return Task.FromCanceled<string>(cancellationToken);
    return Task.FromResult<string>("Done");
}

In [7]:
using System.Diagnostics;
using System.Threading;

var cts = new CancellationTokenSource(TimeSpan.FromSeconds(3));
var token = cts.Token;
//传入委托方法做善后工作
token.Register(() => Console.WriteLine("Cancellation request in Main Thread"));

var sw = Stopwatch.StartNew();
try
{
    //传入委托方法做善后工作
    token.Register(() => Console.WriteLine("Cancellation request in Try Block"));
    await Task.Delay(10000,token);
}
catch(TaskCanceledException ex)
{
    Console.WriteLine("TaskCanceled");
}finally
{
    cts.Dispose();
}

Console.WriteLine($"Task completed in {sw.ElapsedMilliseconds}ms");

Cancellation request in Try Block
TaskCanceled
Task completed in 3009ms
Cancellation request in Main Thread


### 为Task.Run传入CancellationToken